In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Generate some random data for a linear regression problem
np.random.seed(0)
X = 2 * np.random.rand(100, 1)
y = 1 + 2 * X + np.random.randn(100, 1)

# Convert the NumPy arrays to PyTorch tensors
X_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y).float()

In [3]:
# Define a linear regression model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(1, 1)  # One input feature, one output

    def forward(self, x):
        return self.linear(x)

# Instantiate the model and specify a loss function and optimizer
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 1.0176
Epoch [200/1000], Loss: 1.0042
Epoch [300/1000], Loss: 0.9986
Epoch [400/1000], Loss: 0.9957
Epoch [500/1000], Loss: 0.9941
Epoch [600/1000], Loss: 0.9933
Epoch [700/1000], Loss: 0.9929
Epoch [800/1000], Loss: 0.9927
Epoch [900/1000], Loss: 0.9926
Epoch [1000/1000], Loss: 0.9925


In [4]:
# save model as a torchscript model
torch.jit.save(torch.jit.script(model), '../model_repository/linear_regression_model/1/model.pt')

In [7]:
loaded_model = torch.jit.load('../model_repository/linear_regression_model/1/model.pt')
loaded_model.eval();

In [8]:
# sample inference
test = torch.tensor([[2.5]])
pred = loaded_model(test)
pred

tensor([[6.1619]], grad_fn=<AddmmBackward0>)

In [9]:
pred.shape

torch.Size([1, 1])

In [10]:
test.shape

torch.Size([1, 1])

In [1]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification

/home/hacene/miniconda3/envs/deployment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"

class HFWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        out = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return out.logits  # <-- return a Tensor, not a dict


def main():
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
    model.eval()

    wrapped = HFWrapper(model)

    example_input_ids = torch.randint(0, 100, (1, 128))
    example_attention_mask = torch.ones((1, 128), dtype=torch.long)

    traced = torch.jit.trace(
        wrapped,
        (example_input_ids, example_attention_mask)
    )
    traced.save("../model_repository/torch_classifier/1/model.pt")
    print("saved model.pt")
main()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.
/home/hacene/miniconda3/envs/deployment/lib/python3.11/site-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask


saved model.pt


In [7]:
import torchvision
import torch

In [8]:
weights=torchvision.models.resnet.ResNet50_Weights.IMAGENET1K_V1
resnet50 = torchvision.models.resnet50(weights=weights)

In [9]:
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
weights.transforms

functools.partial(<class 'torchvision.transforms._presets.ImageClassification'>, crop_size=224)

In [12]:
# convert to torchscript
example_input = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(resnet50, example_input)
traced_script_module.save("../model_repository/resnet50/1/model.pt")